# Assignment2 - Supervised Learning flow

# Part 1(a) Student details:
* Please write the First-Name, First letter of Last-Name and last 4 digits of the i.d. for each student. 

In [ ]:
# stundent details example: John S. 9812
#                       student details 1: Ilay W. 0725
# (if exists)           student details 2: Aharoni C. 1047
# (if exists)           student details 3: Adir B.
# (if exists&premitted) student details 4: 

## Part 1(b) - Chat-GPT/other AI-agent/other assistance used:
* If you changed the prompt until you got a satisfying answer, please add all versions
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add information:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>   
AI agent name:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 


AI agent name 2:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 

Other assistanse:    
</pre>

## Part 1(c) - Learning Problem and dataset explaination.
* Please explain in one paragraph
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add explaining text:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>




    
</pre>

## Part 2 - Initial Preparations 
You could add as many code cells as needed

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    f1_score,
    make_scorer,
    recall_score,
    precision_score,
    confusion_matrix,
    classification_report
)


In [ ]:
train_df = pd.read_csv('titanic_train.csv')
test_df = pd.read_csv('titanic_test.csv')

print('First 5 training rows:')
print(train_df.head())
print('\nFirst 5 test rows:')
print(test_df.head())



In [ ]:
sns.set(style='whitegrid')

# Survival counts by passenger class
plt.figure(figsize=(6, 4))
sns.countplot(data=train_df, x='Pclass', hue='Survived')
plt.title('Survival count by Passenger Class')
plt.savefig('eda_pclass.png')


# Age distribution for survived vs. not survived passengers
plt.figure(figsize=(6, 4))
sns.histplot(data=train_df, x='Age', hue='Survived', kde=True)
plt.title('Age distribution by survival')
plt.savefig('eda_age.png')


# Correlation heatmap between the numeric features
plt.figure(figsize=(6, 4))
cor = train_df.drop('Survived', axis=1).corr()
sns.heatmap(cor, annot=True, cmap='coolwarm')
plt.title('Feature correlation')
plt.savefig('eda_corr.png')


## Part 3 - Experiments
You could add as many code cells as needed

In [ ]:
# First we need to check if there is any missing data
print(train_df.isnull().sum())
print(test_df.isnull().sum())
# For the scaling im dividing all the data into what needs to be scaled and how
# Categorical data with OneHotEncoding, Numerical data with StandardScaler, and what doesn't need any scaling will pass through the pipeline

numerical_autoscale = ['Age' , 'Fare']
numerical_passthrough = ['SibSp' , 'Parch']
categorical_autoscale = ['Pclass' , 'Sex' , 'Embarked']

 # Because pclass, sex and embarked all are displayed as integers, LogisticReggression Might mistake them as having actuall numerical value which they don't
 # So turning them to strings and passing them trough the OneHotEncoder would make sure they are not treated that way

train_df['Sex'] = train_df['Sex'].map({0 : '0' , 1 : '1'})
train_df['Embarked'] = train_df['Embarked'].map({0: '0' , 1: '1' ,2: '2'})
train_df['Pclass'] = train_df['Pclass'].map({1: '1' , 2: '2' ,3: '3' })


numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')) ,
    ('scaler' , OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numerical', numerical_transformer, numerical_autoscale),
    ('categorical', categorical_transformer, categorical_autoscale),
    ('pass' , 'passthrough' , numerical_passthrough)
])


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier' , LogisticRegression())
])

In [ ]:
grid_params = [
    {
        'classifier' : [LogisticRegression(max_iter=1000)],
        'classifier__C' : [0.1 , 1.0 , 10.0]

    },
    {
        'classifier' : [KNeighborsClassifier()],
        'classifier__n_neighbors' : [3 , 5 , 7]
    },
    {
        'classifier' : [RandomForestClassifier()],
        'classifier__n_estimators' : [50,100],
        'classifier__max_depth' : [5,10,None]
    }}
]

## Part 4 - Training 
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 3)

## Part 5 - Apply on test and show model performance estimation

In [ ]:
X_test = test_df.drop('Survived', axis=1)
y_test = test_df['Survived']
y_pred = best_model.predict(X_test)

test_confusion_matrix = confusion_matrix(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
print('\nTest Confusion matrix:', test_confusion_matrix)
print('\nTest Precision score:', test_precision)
print('\nTest Recall score:', test_recall)
print('\nTest F1 score:', test_f1)

print('\nFirst 5 predictions:')
print(pd.DataFrame({'Actual': y_test[:5].values,
                    'Predicted': y_pred[:5]}))